
# Sample for KFServing SDK

KFServing SDK 샘플입니다. 

interenceService를 만들고 가져오고 학습한 모델을 배포하고 삭제하는 예제가 있는 노트북입니다. 


In [29]:
from kubernetes import client

from kfserving import KFServingClient
from kfserving import constants
from kfserving import utils
from kfserving import V1alpha2EndpointSpec
from kfserving import V1alpha2PredictorSpec
from kfserving import V1alpha2TensorflowSpec
from kfserving import V1alpha2InferenceServiceSpec
from kfserving import V1alpha2InferenceService
from kubernetes.client import V1ResourceRequirements

infereceService를 배포할 네임스페이스를 정합니다. 
아레는 default namespace를 지정합니다. 

In [30]:
namespace = utils.get_default_target_namespace()


# Define InferenceService

먼저 default endpoint spec을 정의합니다. 그리고 inferenceservice의 기본적인 spec을 정의합니다.  


In [31]:
api_version = constants.KFSERVING_GROUP + '/' + constants.KFSERVING_VERSION
default_endpoint_spec = V1alpha2EndpointSpec(
                          predictor=V1alpha2PredictorSpec(
                            tensorflow=V1alpha2TensorflowSpec(
                              storage_uri='pvc://workspace-lecture-tf/saved_model',
                              resources=V1ResourceRequirements(
                                  requests={'cpu':'100m','memory':'1Gi'},
                                  limits={'cpu':'100m', 'memory':'1Gi'}))))
    
isvc = V1alpha2InferenceService(api_version=api_version,
                          kind=constants.KFSERVING_KIND,
                          metadata=client.V1ObjectMeta(
                              name='mnist-server', namespace=namespace),
                          spec=V1alpha2InferenceServiceSpec(default=default_endpoint_spec))


# Create InferenceService

inferenceSerivce를 만들기 위해 KFServingClient를 call합니다.


In [32]:
KFServing = KFServingClient()
KFServing.create(isvc)

{'apiVersion': 'serving.kubeflow.org/v1alpha2',
 'kind': 'InferenceService',
 'metadata': {'creationTimestamp': '2020-09-26T21:15:50Z',
  'generation': 1,
  'name': 'mnist-server',
  'namespace': 'koock',
  'resourceVersion': '3024488',
  'selfLink': '/apis/serving.kubeflow.org/v1alpha2/namespaces/koock/inferenceservices/mnist-server',
  'uid': '705d807e-3a50-4e4d-8f9e-1354b322f03d'},
 'spec': {'default': {'predictor': {'tensorflow': {'resources': {'limits': {'cpu': '100m',
       'memory': '1Gi'},
      'requests': {'cpu': '100m', 'memory': '1Gi'}},
     'runtimeVersion': '1.14.0',
     'storageUri': 'pvc://workspace-lecture-tf/saved_model'}}}},
 'status': {}}

# Check the InferenceService

In [33]:
KFServing.get('mnist-server', namespace=namespace, watch=True)

NAME                 READY      DEFAULT_TRAFFIC CANARY_TRAFFIC  URL                                               
mnist-server         False                                                                                        
mnist-server         False                                                                                        
mnist-server         True       100                             http://mnist-server.koock.example.com/v1/models...


In [34]:
!saved_model_cli show --dir ./saved_model/001 --tag_set serve --signature_def serving_default

The given SavedModel SignatureDef contains the following input(s):
  inputs['flatten_input'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 28, 28)
      name: serving_default_flatten_input:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['dense_1'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 10)
      name: StatefulPartitionedCall:0
Method name is: tensorflow/serving/predict


# Request

# delete

In [ ]:
KFServing.delete('mnist-server', namespace=namespace)